The main idea of dense-net architecture is that each layer receives not only previous layer output, but an output from ALL 
previous layers:
    
layer_1(x) -> x1 -> relu(x1) -> layer_2([x, x1]) -> x2 -> relu(x2) -> layer3([x, x1, x3]) -> x3 ...

In each new layer old outputs will be concatenated. 

In [1]:
from random import random

import torch 
from torch import nn
import torch.nn.functional as F

In [241]:
# growth_rate - means value with which we will increase the output of the layer 
class DenseLayer(nn.Module):
    def __init__(self, in_channels, growth_rate):
        super(DenseLayer, self).__init__()
        self.conv = nn.Conv2d(in_channels, growth_rate, kernel_size=3, padding=1)

    def forward(self, x):
        out = self.conv(x)
        new_features = F.relu(out)
        # print(f"1st: {x.shape}, 2nd: {new_features.shape}")
        out = torch.concat([x, new_features], dim=1) 
        # print(f"concat: {out.shape}")
        return out
    



In [ ]:
# Concatination of tensors:

# x = [3, 224, 224]
# y = [29, 224, 224]
# res = torch.concat([x, y])

# res = [32, 224, 224]

In [243]:
layer1 = DenseLayer(in_channels = 3, growth_rate = 29)
layer2 = DenseLayer(in_channels = 32, growth_rate = 32)

tensor = torch.rand(1, 3, 224, 224)

out = layer1(tensor)
out = layer2(out)
out.shape

torch.Size([1, 64, 224, 224])

So, instead of output_channels using, we assign a growth_rate, which we will add to channel dimension of each output tensor:

tensor([3, 224, 224]) -> conv1(in_channels = 3, growth = 13) -> tensor([(3+13), 224, 224])



In [244]:
# to automatize and unite some separate dense-net layer, we may use dense-block:
class DenseBlock(nn.Module):
    def __init__(self, in_channels, growth_rate, num_layers, pool = True):
        super(DenseBlock, self).__init__()
        # this will contain layers in a list-form 
        
        if pool == True:
            self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        else:
            self.pool = None
            
        self.layers = nn.ModuleList()
        for i in range(num_layers):         # 3           2
            self.layers.append(DenseLayer(in_channels, growth_rate))
            # print(f"{i}: {in_channels} {growth_rate}")
            # previous layer generates output with concatenation and this string make next layer with same output:
            
            # in_channels - 5
            # growth_rate - 3
            # res = concat((5, _, _), (3, _, _)) 
            # res = 8, _, _
            
            # in_channels += growth_rate -> in_channels = 5 + 3 -> 8
            in_channels += growth_rate

    def forward(self, x):
        for layer in self.layers:
            x = layer(x) 
            
            if self.pool:
                x = self.pool(x)
        return x
    


In [245]:
block = DenseBlock(in_channels = 3, growth_rate = 16, num_layers = 4)

In [249]:
tensor = torch.rand(1, 3, 224, 224)

In [250]:
out = block(tensor)
out.shape

torch.Size([1, 67, 14, 14])

In [279]:
class MiniDenseNet(nn.Module):
    def __init__(self, classes=5):
        super(MiniDenseNet, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.block1 = DenseBlock(in_channels = 32, growth_rate = 16, num_layers = 4)
        self.block2 = DenseBlock(in_channels = 96, growth_rate = 16, num_layers = 2, pool = False)
        self.conv2 = nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size=3, padding=1)
        # 32
            
        self.flatten = nn.Flatten()
           
            
        # Финальный слой классификации
        self.linear1 = nn.Linear(512, 64)
        self.linear2 = nn.Linear(64, classes)
             
    def forward(self, x):
        out = self.conv1(x)
        out = F.relu(out)
        out = self.pool(out)
        
        out = self.block1(out)
        out = self.block2(out)
        
        out = self.conv2(out)
        out = F.relu(out)
        out = self.pool(out)
        
        out = self.conv3(out)
        out = F.relu(out)
        out = self.pool(out)
        
        out = self.flatten(out)
        
        out = self.linear1(out)
        out = nn.Dropout(0.5)(out)
        out = F.relu(out)
        
        out = self.linear2(out)
        
        return out
    
    
    def predict(self, x):
        self.eval()
        
        with torch.no_grad():
            
            out = self.forward(x)
            t_out = torch.softmax(out, dim = 1)
            res = torch.argmax(t_out, dim= 1)
            
            return res
        

In [280]:
from torchsummary import summary

test_block = MiniDenseNet()
summary(test_block, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 224, 224]             896
         MaxPool2d-2         [-1, 32, 112, 112]               0
            Conv2d-3         [-1, 16, 112, 112]           4,624
        DenseLayer-4         [-1, 48, 112, 112]               0
         MaxPool2d-5           [-1, 48, 56, 56]               0
            Conv2d-6           [-1, 16, 56, 56]           6,928
        DenseLayer-7           [-1, 64, 56, 56]               0
         MaxPool2d-8           [-1, 64, 28, 28]               0
            Conv2d-9           [-1, 16, 28, 28]           9,232
       DenseLayer-10           [-1, 80, 28, 28]               0
        MaxPool2d-11           [-1, 80, 14, 14]               0
           Conv2d-12           [-1, 16, 14, 14]          11,536
       DenseLayer-13           [-1, 96, 14, 14]               0
        MaxPool2d-14             [-1, 9

In [230]:
test_block = MiniDenseNet()
tensor = torch.rand(3, 224, 224).unsqueeze(0)

res = test_block(tensor)
res.shape

torch.Size([1, 128, 3, 3])